In [1]:
import os
import glob

# from src.envs import TradingEnv
from src.envs.environment_v2 import TradingEnv
from src.utils import DataLoader, Preprocessor, backtest

Loading BokehJS ...

In [2]:
if len(glob.glob(f"./data/^N225/*.csv")) != 0:
    data = DataLoader.load_data(f"./data/^N225/ohlcv.csv")
    features = DataLoader.load_data(f"./data/^N225/features.csv")
else:
    os.makedirs(f"./data/^N225", exist_ok=True)
    data = DataLoader.fetch_data("^N225", interval="1d")
    data, features = Preprocessor.extract_features(data)
    data.to_csv(f"./data/^N225/ohlcv.csv")
    features.to_csv(f"./data/^N225/features.csv")

data = data.iloc[-50:]
features = features.iloc[-50:]

data_train, features_train, data_eval, features_eval = Preprocessor.train_test_split(data, features, test_size=0.5)

In [3]:
from src.envs.actions import BuySell
env = TradingEnv(data_eval, features_eval, debug=True, stop_loss=True, actions=BuySell)

In [4]:
actions = env.actions
actions.BUY

<BuySell.BUY: 1>

In [7]:
obs = env.reset()
done = False
while not done:
    # obs, _, done, _ = env.step(env.action_space.sample())
    obs, _, done, _ = env.step(0)
    print(obs[:, -4:])
    for order in env.orders:
        print(env.position)
        print(order.stop)
        # print(order.stop_price)

=============== Environment (2021-10-19 00:00:00) ===============
Price: 29215.51953125
Assets: 100000
Equity: 100000
Orders: [<Order size=-3.0, sl=29272.49023, contingent=0>]
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 0, Reward: 0, Done: False

[[-1.6943163   0.          0.          0.        ]
 [-1.60370038  0.          0.          0.        ]
 [-1.45267385  0.          0.          0.        ]
 [-1.82520596  0.          0.          0.        ]
 [-2.32862773  0.          0.          0.        ]]
=============== Environment (2021-10-20 00:00:00) ===============
Price: 29255.55078125
Assets: 99741.44133203125
Equity: 99741.44133203125
Orders: [<Order size=-3.0, sl=29489.10938, contingent=0>]
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: [<Trade size=-3 time=5-6 price=29186.30401171875-29272.490234375 pnl=-259>]
Action: 0, Reward: -0.002588935081895727, Done: False

[[-1.60370038  0.          0.          0.        ]
 [-1.45267385  0.          0

In [6]:
# backtest(env)
backtest(env, agent="Random")
# results = backtest(env, agent="Buy&Hold", save_dir="test", plot=True)
# results = backtest(env, agent="Sell&Hold", save_dir="test", plot=True)
# results

=============== Environment (2021-10-19 00:00:00) ===============
Price: 29215.51953125
Assets: 100000
Equity: 100000
Orders: [<Order size=-3.0, sl=29272.49023, contingent=0>]
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: []
Action: 0, Reward: 0, Done: False

=============== Environment (2021-10-20 00:00:00) ===============
Price: 29255.55078125
Assets: 99741.44133203125
Equity: 99741.44133203125
Orders: [<Order size=-3.0, sl=29489.10938, contingent=0>]
Trades: []
Position: <Position: 0 (0 trades)>
Closed Trades: [<Trade size=-3 time=5-6 price=29186.30401171875-29272.490234375 pnl=-259>]
Action: 0, Reward: -0.002588935081895727, Done: False

=============== Environment (2021-10-21 00:00:00) ===============
Price: 28708.580078125
Assets: 99741.44133203125
Equity: 101294.5867890625
Orders: [<Order size=3.0, stop=29489.10938, contingent=1>]
Trades: [<Trade size=-3 time=6- price=29226.29523046875- pnl=1553>]
Position: <Position: -3 (1 trades)>
Closed Trades: [<Trade size=-3 

Start                     2021-10-12 00:00:00
End                       2021-11-16 00:00:00
Duration                     35 days 00:00:00
Exposure Time [%]                        72.0
Equity Final [$]                104123.025906
Equity Peak [$]                 104123.025906
Return [%]                           4.123026
Buy & Hold Return [%]                5.587941
Return (Ann.) [%]                   50.269745
Volatility (Ann.) [%]                11.03189
Sharpe Ratio                          3.90129
Sortino Ratio                        9.548515
Calmar Ratio                          33.6786
Max. Drawdown [%]                   -1.492632
Avg. Drawdown [%]                    -0.63743
Max. Drawdown Duration        7 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                       2.766725
Worst Trade [%]                     -0.594922
Avg. Trade [%]                    